1. M1 비중 낮추기

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

# 경로 설정 (모두 submissions 폴더)
DATA_PATH = "../../data/raw/"
SUB_PATH = "./submissions/"

print("="*60)
print("파일 로드 중...")
print("="*60)

# 1. 정답 로드 (m7 필터링 기준)
train_df = pd.read_csv(f"{DATA_PATH}train.csv")
m7_mask = (train_df.familysize <= 50)
a_cols = [col for col in train_df.columns if col.endswith('A') and col.startswith('Q')]
m7_mask &= (train_df[a_cols].std(axis=1) > 0)
m7_indices = train_df[m7_mask].index
train_y = (2 - train_df.loc[m7_indices, 'voted'].to_numpy()).astype(np.float32)

# 2. OOF 로드 (submissions 폴더에서)
m1_oof_raw = np.load(f"{SUB_PATH}exp11_sota_AUC_0.77212.npy")
m1_indices = train_df[train_df.familysize <= 50].index
m1_oof = pd.DataFrame(m1_oof_raw, index=m1_indices).reindex(m7_indices).values.flatten()

m7_oof = np.load(f"{SUB_PATH}exp25_m7_Refined_AUC_0.77375.npy").flatten()

# 3. Rank 변환
m1_rank = pd.Series(m1_oof).rank(pct=True).values
m7_rank = pd.Series(m7_oof).rank(pct=True).values

print(f"✅ OOF 로드 완료")
print(f"   m1 OOF AUC: {roc_auc_score(train_y, m1_oof):.5f}")
print(f"   m7 OOF AUC: {roc_auc_score(train_y, m7_oof):.5f}")
print(f"   상관계수: {np.corrcoef(m1_rank, m7_rank)[0,1]:.4f}\n")

# 4. Rank 비율 미세 조정
print("="*60)
print("Rank Averaging 비율 최적화")
print("="*60)

weights = [round(w, 2) for w in np.arange(0.1, 0.99, 0.01)]


results = []
for w in weights:
    ensemble_rank = m1_rank * w + m7_rank * (1-w)
    auc = roc_auc_score(train_y, ensemble_rank)
    
    results.append({
        'm1_weight': w,
        'm7_weight': 1-w,
        'oof_auc': auc
    })
    
    print(f"{w:.2f} : {1-w:.2f} → OOF AUC: {auc:.5f}")

df_results = pd.DataFrame(results).sort_values('oof_auc', ascending=False)

print("\n" + "="*60)
print("Top 3 조합")
print("="*60)
print(df_results.head(3).to_string(index=False))

# 5. 최고 조합으로 테스트 데이터 앙상블
best = df_results.iloc[0]
best_w = best['m1_weight']

print("\n" + "="*60)
print(f"🏆 최고 조합: {best_w:.2f} : {1-best_w:.2f}")
print(f"   OOF AUC: {best['oof_auc']:.5f}")
print("="*60)

# 6. 테스트 데이터 Rank Averaging
m1_test = pd.read_csv(f"{SUB_PATH}0130-1923.csv")['voted']
m7_test = pd.read_csv(f"{SUB_PATH}m7_Refined_test_preds.csv")['voted']

# m1을 정방향으로 복구
m1_test_fixed = 2.0 - m1_test
m1_t_rank = m1_test_fixed.rank(pct=True).values
m7_t_rank = m7_test.rank(pct=True).values

# 최고 비율로 앙상블
final_test_rank = m1_t_rank * best_w + m7_t_rank * (1-best_w)

파일 로드 중...
✅ OOF 로드 완료
   m1 OOF AUC: 0.77196
   m7 OOF AUC: 0.77375
   상관계수: 0.9852

Rank Averaging 비율 최적화
0.10 : 0.90 → OOF AUC: 0.77386
0.11 : 0.89 → OOF AUC: 0.77386
0.12 : 0.88 → OOF AUC: 0.77387
0.13 : 0.87 → OOF AUC: 0.77388
0.14 : 0.86 → OOF AUC: 0.77388
0.15 : 0.85 → OOF AUC: 0.77389
0.16 : 0.84 → OOF AUC: 0.77389
0.17 : 0.83 → OOF AUC: 0.77389
0.18 : 0.82 → OOF AUC: 0.77390
0.19 : 0.81 → OOF AUC: 0.77390
0.20 : 0.80 → OOF AUC: 0.77390
0.21 : 0.79 → OOF AUC: 0.77390
0.22 : 0.78 → OOF AUC: 0.77390
0.23 : 0.77 → OOF AUC: 0.77390
0.24 : 0.76 → OOF AUC: 0.77390
0.25 : 0.75 → OOF AUC: 0.77390
0.26 : 0.74 → OOF AUC: 0.77389
0.27 : 0.73 → OOF AUC: 0.77389
0.28 : 0.72 → OOF AUC: 0.77389
0.29 : 0.71 → OOF AUC: 0.77388
0.30 : 0.70 → OOF AUC: 0.77388
0.31 : 0.69 → OOF AUC: 0.77387
0.32 : 0.68 → OOF AUC: 0.77386
0.33 : 0.67 → OOF AUC: 0.77385
0.34 : 0.66 → OOF AUC: 0.77385
0.35 : 0.65 → OOF AUC: 0.77384
0.36 : 0.64 → OOF AUC: 0.77383
0.37 : 0.63 → OOF AUC: 0.77382
0.38 : 0.62 → OOF AUC: 0

In [7]:
# m1 기준(Small = Voted)으로 변환
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")
submission['voted'] = 2.0 - final_test_rank

# 저장
output_name = f"{SUB_PATH}Optimized_m1m7_{best_w:.0%}_{1-best_w:.0%}_AUC{best['oof_auc']:.5f}.csv"
submission.to_csv(output_name, index=False)

print(f"\n✅ 최종 파일 저장: {output_name}")
print(f"   예상 LB 점수: 0.7815+ (기존 0.50:0.50 대비 개선)")

# 7. Top 3 모두 저장
print("\n" + "="*60)
print("Top 3 조합 모두 저장")
print("="*60)

for idx, row in df_results.head(3).iterrows():
    w = row['m1_weight']
    final_rank = m1_t_rank * w + m7_t_rank * (1-w)
    
    sub = pd.read_csv(f"{DATA_PATH}sample_submission.csv")
    sub['voted'] = 2.0 - final_rank
    
    fname = f"{SUB_PATH}m1m7_{w:.0%}_{1-w:.0%}_AUC{row['oof_auc']:.5f}.csv"
    sub.to_csv(fname, index=False)
    print(f"✅ {fname}")

print("\n🎯 완료! Top 3 파일 제출해보세요!")


✅ 최종 파일 저장: ./submissions/Optimized_m1m7_21%_79%_AUC0.77390.csv
   예상 LB 점수: 0.7815+ (기존 0.50:0.50 대비 개선)

Top 3 조합 모두 저장
✅ ./submissions/m1m7_21%_79%_AUC0.77390.csv
✅ ./submissions/m1m7_23%_77%_AUC0.77390.csv
✅ ./submissions/m1m7_22%_78%_AUC0.77390.csv

🎯 완료! Top 3 파일 제출해보세요!
